# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f66d87eb2e0>
├── 'a' --> tensor([[-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756]])
└── 'x' --> <FastTreeValue 0x7f66aff75160>
    └── 'c' --> tensor([[ 0.5222, -0.0936, -0.3602, -1.5169],
                        [ 0.1019,  0.9881, -0.6978,  0.7938],
                        [ 0.1230,  1.6142, -0.8124,  0.9983]])

In [4]:
t.a

tensor([[-0.5305,  0.0568,  2.2270],
        [-0.3141,  0.0712,  0.9756]])

In [5]:
%timeit t.a

63.3 ns ± 0.0413 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f66d87eb2e0>
├── 'a' --> tensor([[-0.1896, -0.7142,  0.7458],
│                   [ 0.0411,  2.1006, -0.5813]])
└── 'x' --> <FastTreeValue 0x7f66aff75160>
    └── 'c' --> tensor([[ 0.5222, -0.0936, -0.3602, -1.5169],
                        [ 0.1019,  0.9881, -0.6978,  0.7938],
                        [ 0.1230,  1.6142, -0.8124,  0.9983]])

In [7]:
%timeit t.a = new_value

58.8 ns ± 0.0229 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5305,  0.0568,  2.2270],
               [-0.3141,  0.0712,  0.9756]]),
    x: Batch(
           c: tensor([[ 0.5222, -0.0936, -0.3602, -1.5169],
                      [ 0.1019,  0.9881, -0.6978,  0.7938],
                      [ 0.1230,  1.6142, -0.8124,  0.9983]]),
       ),
)

In [10]:
b.a

tensor([[-0.5305,  0.0568,  2.2270],
        [-0.3141,  0.0712,  0.9756]])

In [11]:
%timeit b.a

59.6 ns ± 0.0207 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.4810, -0.8100,  1.1806],
               [ 1.0253, -1.4430, -0.1589]]),
    x: Batch(
           c: tensor([[ 0.5222, -0.0936, -0.3602, -1.5169],
                      [ 0.1019,  0.9881, -0.6978,  0.7938],
                      [ 0.1230,  1.6142, -0.8124,  0.9983]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.105 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

849 ns ± 0.132 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 26.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 2.96 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 610 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f66acc74130>
├── 'a' --> tensor([[[-0.5305,  0.0568,  2.2270],
│                    [-0.3141,  0.0712,  0.9756]],
│           
│                   [[-0.5305,  0.0568,  2.2270],
│                    [-0.3141,  0.0712,  0.9756]],
│           
│                   [[-0.5305,  0.0568,  2.2270],
│                    [-0.3141,  0.0712,  0.9756]],
│           
│                   [[-0.5305,  0.0568,  2.2270],
│                    [-0.3141,  0.0712,  0.9756]],
│           
│                   [[-0.5305,  0.0568,  2.2270],
│                    [-0.3141,  0.0712,  0.9756]],
│           
│                   [[-0.5305,  0.0568,  2.2270],
│                    [-0.3141,  0.0712,  0.9756]],
│           
│                   [[-0.5305,  0.0568,  2.2270],
│                    [-0.3141,  0.0712,  0.9756]],
│           
│                   [[-0.5305,  0.0568,  2.2270],
│                    [-0.3141,  0.0712,  0.9756]]])
└── 'x' --> <FastTreeValue 0x7f66acc74940>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 76.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f662ff80a90>
├── 'a' --> tensor([[-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756],
│                   [-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756],
│                   [-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756],
│                   [-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756],
│                   [-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756],
│                   [-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756],
│                   [-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756],
│                   [-0.5305,  0.0568,  2.2270],
│                   [-0.3141,  0.0712,  0.9756]])
└── 'x' --> <FastTreeValue 0x7f662ff804c0>
    └── 'c' --> tensor([[ 0.5222, -0.0936, -0.3602, -1.5169],
                        [ 0.1019,  0.9881, -0.6978,  0.7938],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 296 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 56.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5222, -0.0936, -0.3602, -1.5169],
                       [ 0.1019,  0.9881, -0.6978,  0.7938],
                       [ 0.1230,  1.6142, -0.8124,  0.9983]],
              
                      [[ 0.5222, -0.0936, -0.3602, -1.5169],
                       [ 0.1019,  0.9881, -0.6978,  0.7938],
                       [ 0.1230,  1.6142, -0.8124,  0.9983]],
              
                      [[ 0.5222, -0.0936, -0.3602, -1.5169],
                       [ 0.1019,  0.9881, -0.6978,  0.7938],
                       [ 0.1230,  1.6142, -0.8124,  0.9983]],
              
                      [[ 0.5222, -0.0936, -0.3602, -1.5169],
                       [ 0.1019,  0.9881, -0.6978,  0.7938],
                       [ 0.1230,  1.6142, -0.8124,  0.9983]],
              
                      [[ 0.5222, -0.0936, -0.3602, -1.5169],
                       [ 0.1019,  0.9881, -0.6978,  0.7938],
                       [ 0.1230,  1.6142, -0.8124,  0.9983]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 162 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5222, -0.0936, -0.3602, -1.5169],
                      [ 0.1019,  0.9881, -0.6978,  0.7938],
                      [ 0.1230,  1.6142, -0.8124,  0.9983],
                      [ 0.5222, -0.0936, -0.3602, -1.5169],
                      [ 0.1019,  0.9881, -0.6978,  0.7938],
                      [ 0.1230,  1.6142, -0.8124,  0.9983],
                      [ 0.5222, -0.0936, -0.3602, -1.5169],
                      [ 0.1019,  0.9881, -0.6978,  0.7938],
                      [ 0.1230,  1.6142, -0.8124,  0.9983],
                      [ 0.5222, -0.0936, -0.3602, -1.5169],
                      [ 0.1019,  0.9881, -0.6978,  0.7938],
                      [ 0.1230,  1.6142, -0.8124,  0.9983],
                      [ 0.5222, -0.0936, -0.3602, -1.5169],
                      [ 0.1019,  0.9881, -0.6978,  0.7938],
                      [ 0.1230,  1.6142, -0.8124,  0.9983],
                      [ 0.5222, -0.0936, -0.3602, -1.5169],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 163 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
